In [3]:
import pypsa, numpy as np

# General information about marginal costs per technology
marginal_costs = {"WindA" : 0,
                  "CoalA" : 30,
                  "GasA" : 60,
                  "OilB" : 80,                  
                  "CoalC" : 30,
                  }

# Power plant capacities (nominal powers in MW) in each node
power_plant_p_nom = {"NodeA" : {"CoalA" : 35000,
                                "WindA" : 3000,
                                "GasA" : 8000,
                               },
                     "NodeB" : {"OilB" : 7000,
                                },
                     "NodeC" : {"CoalC" : 6000,
                               },
                    }


#%% --------------------------------------------|
# First stage - Daily optimal economic dispatch |
# ----------------------------------------------|

country = "Dafoeland"

# Combine all power plants in one
# -------------------------------
all_plants = dict()
                    
for i in power_plant_p_nom.keys():
    all_plants = {**all_plants, **power_plant_p_nom[i]}
    
power_plants = dict()
power_plants[country] = all_plants

# Start PyPSA analysis
# --------------------
network = pypsa.Network()

# Timesteps labelled by [0,1,2,3,...,23]
network.set_snapshots(range(3))

wind_time_series = [0.3,0.6,0.4]

loads = {"Dafoeland" : [42000, 35000, 46000]}

network.add("Bus",country)

for tech in power_plants[country]:
    network.add("Generator",
                "{}".format(tech),
                bus=country,
                p_nom=power_plants[country][tech],
                marginal_cost=marginal_costs[tech],
                p_max_pu=(wind_time_series if tech == "WindA" else 1),
                )

# Load which varies over the snapshots
network.add("Load",
            "{} load".format(country),
            bus=country,
            p_set=loads[country])

network.lopf(network.snapshots, mode = "lopf", solver_name="gurobi")

network.loads_t.p
network.generators_t.p
network.buses_t.marginal_price

#%% --------------------------------|
# Second stage - Optimal redispatch |
# ----------------------------------|
network_redispatch = pypsa.Network()

network_redispatch.set_snapshots(range(3))

# Add three buses
network_redispatch.add("Bus","BusA")
network_redispatch.add("Bus","BusB")
network_redispatch.add("Bus","BusC")

# Add three lines in a ring
network_redispatch.add("Line","My line AB",
            bus0="BusA",
            bus1="BusB",
            x=0.0001,
            s_nom=1000)

network_redispatch.add("Line","My line BC",
            bus0="BusB",
            bus1="BusC",
            x=0.0001,
            s_nom=1000)

network_redispatch.add("Line","My line CA",
            bus0="BusC",
            bus1="BusA",
            x=0.0001,
            s_nom=60000)

#print(network_redispatch.lines)

# Add generators
# --------------

# CoalA
network_redispatch.add("Generator",
            "CoalA",
            bus="BusA",
            p_nom=power_plant_p_nom["NodeA"]["CoalA"],
            marginal_cost=0,
            p_max_pu=(network.generators_t.p["CoalA"] / power_plants['Dafoeland']['CoalA']).tolist(),
            p_min_pu=(network.generators_t.p["CoalA"] / power_plants['Dafoeland']['CoalA']).tolist(),
            )

network_redispatch.add("Generator",
            "CoalA-pos",
            bus="BusA",
            p_nom=power_plant_p_nom["NodeA"]["CoalA"],
            marginal_cost=marginal_costs["CoalA"],
            p_max_pu=(1 - network.generators_t.p["CoalA"] / power_plants['Dafoeland']['CoalA']).tolist(),
            p_min_pu=0,
            )

network_redispatch.add("Generator",
            "CoalA-neg",
            bus="BusA",
            p_nom=power_plant_p_nom["NodeA"]["CoalA"],
            marginal_cost=(marginal_costs["CoalA"] - network.buses_t.marginal_price['Dafoeland']).tolist(),
            p_max_pu=0,
            p_min_pu=(- network.generators_t.p["CoalA"] / power_plants['Dafoeland']['CoalA']).tolist(),
            )

# GasA
network_redispatch.add("Generator",
            "GasA",
            bus="BusA",
            p_nom=power_plant_p_nom["NodeA"]["GasA"],
            marginal_cost=0,
            p_max_pu=(network.generators_t.p["GasA"] / power_plants['Dafoeland']['GasA']).tolist(),
            p_min_pu=(network.generators_t.p["GasA"] / power_plants['Dafoeland']['GasA']).tolist(),
            )

network_redispatch.add("Generator",
            "GasA-pos",
            bus="BusA",
            p_nom=power_plant_p_nom["NodeA"]["GasA"],
            marginal_cost=marginal_costs["GasA"],
            p_max_pu=(1 - network.generators_t.p["GasA"] / power_plants['Dafoeland']['GasA']).tolist(),
            p_min_pu=0,
            )

network_redispatch.add("Generator",
            "GasA-neg",
            bus="BusA",
            p_nom=power_plant_p_nom["NodeA"]["GasA"],
            marginal_cost=(marginal_costs["GasA"] - network.buses_t.marginal_price['Dafoeland']).tolist(),
            p_max_pu=0,
            p_min_pu=(- network.generators_t.p["GasA"] / power_plants['Dafoeland']['GasA']).tolist(),
            )


# WindA - only negative redispatch? --> curtailment?
network_redispatch.add("Generator",
            "WindA",
            bus="BusA",
            p_nom=power_plant_p_nom["NodeA"]["WindA"],
            marginal_cost=0,
            p_max_pu=(network.generators_t.p["WindA"] / power_plants['Dafoeland']['WindA']).tolist(),
            p_min_pu=(network.generators_t.p["WindA"] / power_plants['Dafoeland']['WindA']).tolist(),
            )

network_redispatch.add("Generator",
            "WindA-neg",
            bus="BusA",
            p_nom=power_plant_p_nom["NodeA"]["WindA"],
            marginal_cost=(marginal_costs["WindA"] - network.buses_t.marginal_price['Dafoeland']).tolist(),
            p_max_pu=0,
            p_min_pu=(- network.generators_t.p["WindA"] / power_plants['Dafoeland']['WindA']).tolist(),
            )


# OilB
network_redispatch.add("Generator",
            "OilB",
            bus="BusB",
            p_nom=power_plant_p_nom["NodeB"]["OilB"],
            marginal_cost=0,
            p_max_pu=(network.generators_t.p["OilB"] / power_plants['Dafoeland']['OilB']).tolist(),
            p_min_pu=(network.generators_t.p["OilB"] / power_plants['Dafoeland']['OilB']).tolist(),
            )

network_redispatch.add("Generator",
            "OilB-pos",
            bus="BusB",
            p_nom=power_plant_p_nom["NodeB"]["OilB"],
            marginal_cost=marginal_costs["OilB"],
            p_max_pu=(1 - network.generators_t.p["OilB"] / power_plants['Dafoeland']['OilB']).tolist(),
            p_min_pu=0,
            )

network_redispatch.add("Generator",
            "OilB-neg",
            bus="BusB",
            p_nom=power_plant_p_nom["NodeB"]["OilB"],
            marginal_cost=(marginal_costs["OilB"] - network.buses_t.marginal_price['Dafoeland']).tolist(),
            p_max_pu=0,
            p_min_pu=(- network.generators_t.p["OilB"] / power_plants['Dafoeland']['OilB']).tolist(),
            )
print(marginal_costs["OilB"])
print(network.buses_t.marginal_price['Dafoeland'])
print((marginal_costs["OilB"] - network.buses_t.marginal_price['Dafoeland']).tolist())

# CoalC
network_redispatch.add("Generator",
            "CoalC",
            bus="BusC",
            p_nom=power_plant_p_nom["NodeC"]["CoalC"],
            marginal_cost=0,
            p_max_pu=(network.generators_t.p["CoalC"] / power_plants['Dafoeland']['CoalC']).tolist(),
            p_min_pu=(network.generators_t.p["CoalC"] / power_plants['Dafoeland']['CoalC']).tolist(),
            )

network_redispatch.add("Generator",
            "CoalC-pos",
            bus="BusC",
            p_nom=power_plant_p_nom["NodeC"]["CoalC"],
            marginal_cost=marginal_costs["CoalC"],
            p_max_pu=(1 - network.generators_t.p["CoalC"] / power_plants['Dafoeland']['CoalC']).tolist(),
            p_min_pu=0,
            )

network_redispatch.add("Generator",
            "CoalC-neg",
            bus="BusC",
            p_nom=power_plant_p_nom["NodeC"]["CoalC"],
            marginal_cost=(marginal_costs["CoalC"] - network.buses_t.marginal_price['Dafoeland']).tolist(),
            p_max_pu=0,
            p_min_pu=(- network.generators_t.p["CoalC"] / power_plants['Dafoeland']['CoalC']).tolist(),
            )

# Add loads
# ---------

loads = {"NodeA" : [32000, 25000, 36000],
         "NodeB" : [7000, 7000, 7000],
         "NodeC" : [3000, 3000, 3000]
        }


network_redispatch.add("Load",
            "LoadA",
            bus="BusA",
            p_set=loads["NodeA"])

network_redispatch.add("Load",
            "LoadB",
            bus="BusB",
            p_set=loads["NodeB"])

network_redispatch.add("Load",
            "LoadC",
            bus="BusC",
            p_set=loads["NodeC"])
#print(network_redispatch.generators)
# Solve problem
# -------------
network_redispatch.lopf(network.snapshots, solver_name="gurobi")
network_redispatch.generators_t.p

##%% -----------------------------------------------|
## Second stage - Optimal redispatch with batteries |
## -------------------------------------------------|
#network_redispatch_w_bat = pypsa.Network()
#
#network_redispatch_w_bat.set_snapshots(range(3))
#
## Add three buses
#network_redispatch_w_bat.add("Bus","BusA")
#network_redispatch_w_bat.add("Bus","BusB")
#network_redispatch_w_bat.add("Bus","BusC")
#
## Add three lines in a ring
#network_redispatch_w_bat.add("Line","My line AB",
#            bus0="BusA",
#            bus1="BusB",
#            x=0.0001,
#            s_nom=1000)
#
#network_redispatch_w_bat.add("Line","My line BC",
#            bus0="BusB",
#            bus1="BusC",
#            x=0.0001,
#            s_nom=1000)
#
#network_redispatch_w_bat.add("Line","My line CA",
#            bus0="BusC",
#            bus1="BusA",
#            x=0.0001,
#            s_nom=60000)
#
#print(network_redispatch_w_bat.lines)
#
## Add generators
## --------------
#
## CoalA
#network_redispatch_w_bat.add("Generator",
#            "CoalA",
#            bus="BusA",
#            p_nom=power_plant_p_nom["NodeA"]["CoalA"],
#            marginal_cost=0,
#            p_max_pu=(network.generators_t.p["CoalA"] / power_plants['Dafoeland']['CoalA']).tolist(),
#            p_min_pu=(network.generators_t.p["CoalA"] / power_plants['Dafoeland']['CoalA']).tolist(),
#            )
#
#network_redispatch_w_bat.add("Generator",
#            "CoalA-pos",
#            bus="BusA",
#            p_nom=power_plant_p_nom["NodeA"]["CoalA"],
#            marginal_cost=marginal_costs["CoalA"],
#            p_max_pu=(1 - network.generators_t.p["CoalA"] / power_plants['Dafoeland']['CoalA']).tolist(),
#            p_min_pu=0,
#            )
#
#network_redispatch_w_bat.add("Generator",
#            "CoalA-neg",
#            bus="BusA",
#            p_nom=power_plant_p_nom["NodeA"]["CoalA"],
#            marginal_cost=(marginal_costs["CoalA"] - network.buses_t.marginal_price['Dafoeland']).tolist(),
#            p_max_pu=0,
#            p_min_pu=(- network.generators_t.p["CoalA"] / power_plants['Dafoeland']['CoalA']).tolist(),
#            )
#
## GasA
#network_redispatch_w_bat.add("Generator",
#            "GasA",
#            bus="BusA",
#            p_nom=power_plant_p_nom["NodeA"]["GasA"],
#            marginal_cost=0,
#            p_max_pu=(network.generators_t.p["GasA"] / power_plants['Dafoeland']['GasA']).tolist(),
#            p_min_pu=(network.generators_t.p["GasA"] / power_plants['Dafoeland']['GasA']).tolist(),
#            )
#
#network_redispatch_w_bat.add("Generator",
#            "GasA-pos",
#            bus="BusA",
#            p_nom=power_plant_p_nom["NodeA"]["GasA"],
#            marginal_cost=marginal_costs["GasA"],
#            p_max_pu=(1 - network.generators_t.p["GasA"] / power_plants['Dafoeland']['GasA']).tolist(),
#            p_min_pu=0,
#            )
#
#network_redispatch_w_bat.add("Generator",
#            "GasA-neg",
#            bus="BusA",
#            p_nom=power_plant_p_nom["NodeA"]["GasA"],
#            marginal_cost=(marginal_costs["GasA"] - network.buses_t.marginal_price['Dafoeland']).tolist(),
#            p_max_pu=0,
#            p_min_pu=(- network.generators_t.p["GasA"] / power_plants['Dafoeland']['GasA']).tolist(),
#            )
#
#
## WindA - only negative redispatch? --> curtailment?
#network_redispatch_w_bat.add("Generator",
#            "WindA",
#            bus="BusA",
#            p_nom=power_plant_p_nom["NodeA"]["WindA"],
#            marginal_cost=0,
#            p_max_pu=(network.generators_t.p["WindA"] / power_plants['Dafoeland']['WindA']).tolist(),
#            p_min_pu=(network.generators_t.p["WindA"] / power_plants['Dafoeland']['WindA']).tolist(),
#            )
#
#network_redispatch_w_bat.add("Generator",
#            "WindA-neg",
#            bus="BusA",
#            p_nom=power_plant_p_nom["NodeA"]["WindA"],
#            marginal_cost=(marginal_costs["WindA"] - network.buses_t.marginal_price['Dafoeland']).tolist(),
#            p_max_pu=0,
#            p_min_pu=(- network.generators_t.p["WindA"] / power_plants['Dafoeland']['WindA']).tolist(),
#            )
#
#
## OilB
#network_redispatch_w_bat.add("Generator",
#            "OilB",
#            bus="BusB",
#            p_nom=power_plant_p_nom["NodeB"]["OilB"],
#            marginal_cost=0,
#            p_max_pu=(network.generators_t.p["OilB"] / power_plants['Dafoeland']['OilB']).tolist(),
#            p_min_pu=(network.generators_t.p["OilB"] / power_plants['Dafoeland']['OilB']).tolist(),
#            )
#
#network_redispatch_w_bat.add("Generator",
#            "OilB-pos",
#            bus="BusB",
#            p_nom=power_plant_p_nom["NodeB"]["OilB"],
#            marginal_cost=marginal_costs["OilB"],
#            p_max_pu=(1 - network.generators_t.p["OilB"] / power_plants['Dafoeland']['OilB']).tolist(),
#            p_min_pu=0,
#            )
#
#network_redispatch_w_bat.add("Generator",
#            "OilB-neg",
#            bus="BusB",
#            p_nom=power_plant_p_nom["NodeB"]["OilB"],
#            marginal_cost=(marginal_costs["OilB"] - network.buses_t.marginal_price['Dafoeland']).tolist(),
#            p_max_pu=0,
#            p_min_pu=(- network.generators_t.p["OilB"] / power_plants['Dafoeland']['OilB']).tolist(),
#            )
#
#
## CoalC
#network_redispatch_w_bat.add("Generator",
#            "CoalC",
#            bus="BusC",
#            p_nom=power_plant_p_nom["NodeC"]["CoalC"],
#            marginal_cost=0,
#            p_max_pu=(network.generators_t.p["CoalC"] / power_plants['Dafoeland']['CoalC']).tolist(),
#            p_min_pu=(network.generators_t.p["CoalC"] / power_plants['Dafoeland']['CoalC']).tolist(),
#            )
#
#network_redispatch_w_bat.add("Generator",
#            "CoalC-pos",
#            bus="BusC",
#            p_nom=power_plant_p_nom["NodeC"]["CoalC"],
#            marginal_cost=marginal_costs["CoalC"],
#            p_max_pu=(1 - network.generators_t.p["CoalC"] / power_plants['Dafoeland']['CoalC']).tolist(),
#            p_min_pu=0,
#            )
#
#network_redispatch_w_bat.add("Generator",
#            "CoalC-neg",
#            bus="BusC",
#            p_nom=power_plant_p_nom["NodeC"]["CoalC"],
#            marginal_cost=(marginal_costs["CoalC"] - network.buses_t.marginal_price['Dafoeland']).tolist(),
#            p_max_pu=0,
#            p_min_pu=(- network.generators_t.p["CoalC"] / power_plants['Dafoeland']['CoalC']).tolist(),
#            )
#
## Add storage
## -----------
#network_redispatch_w_bat.add("StorageUnit",
#            "StorageA",
#            bus="BusA",
#            p_nom=10000,
#            max_hours=4, #energy storage in terms of hours at full power
#           )
#
#network_redispatch_w_bat.add("StorageUnit",
#            "StorageB",
#            bus="BusB",
#            p_nom=10000,
#            max_hours=4, #energy storage in terms of hours at full power
#           )
#
#network_redispatch_w_bat.add("StorageUnit",
#            "StorageC",
#            bus="BusC",
#            p_nom=10000,
#            max_hours=4, #energy storage in terms of hours at full power
#           )
#
## Add loads
## ---------
#
#loads = {"NodeA" : [32000, 25000, 36000],
#         "NodeB" : [7000, 7000, 7000],
#         "NodeC" : [3000, 3000, 3000]
#        }
#
#network_redispatch_w_bat.add("Load",
#            "LoadA",
#            bus="BusA",
#            p_set=loads["NodeA"])
#
#network_redispatch_w_bat.add("Load",
#            "LoadB",
#            bus="BusB",
#            p_set=loads["NodeB"])
#
#network_redispatch_w_bat.add("Load",
#            "LoadC",
#            bus="BusC",
#            p_set=loads["NodeC"])
#
## Solve problem
## -------------
#network_redispatch_w_bat.lopf(network.snapshots, solver_name="gurobi")
#
## Results comparison
## ------------------
#dispatch = network.generators_t.p
#redispatch = network_redispatch.generators_t.p
#redispatch_w_bat = network_redispatch_w_bat.generators_t.p
#redispatch_w_bat2 = network_redispatch_w_bat.storage_units_t.p
#

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
c:\users\willem\pypsa-eur\pypsa\pypsa\opf.py:86: FutureWarning:

Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead

INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x16
  Lower bound: 3690000.0
  Upper bound: 3690000.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 16
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 16
  Number of nonzeros: 16
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation


80
0    60.0
1    30.0
2    60.0
Name: Dafoeland, dtype: float64
[20.0, 50.0, 20.0]


c:\users\willem\pypsa-eur\pypsa\pypsa\opf.py:86: FutureWarning:

Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead

INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x52
  Lower bound: 850000.0
  Upper bound: 850000.0
  Number of objectives: 1
  Number of constraints: 31
  Number of variables: 52
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 52
  Number of nonzeros: 88
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality 

,CoalA,CoalA-pos,CoalA-neg,GasA,GasA-pos,GasA-neg,WindA,WindA-neg,OilB,OilB-pos,OilB-neg,CoalC,CoalC-pos,CoalC-neg
0,35000.0,0.0,0.0,100.0,0.0,0.0,900.0,0.0,0.0,3500.0,0.0,6000.0,0.0,-3500.0
1,33200.0,0.0,-1000.0,0.0,0.0,0.0,1800.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0
2,35000.0,0.0,0.0,3800.0,0.0,0.0,1200.0,0.0,0.0,3500.0,0.0,6000.0,0.0,-3500.0


In [16]:
display(network_redispatch.generators_t.p_min_pu)
network_redispatch.generators_t.p_max_pu

,CoalA,CoalA-neg,GasA,GasA-neg,WindA,WindA-neg,OilB,OilB-neg,CoalC,CoalC-neg
0,1.000000,-1.000000,0.0125,-0.0125,0.3,-0.3,0.0,-0.0,1.0,-1.0
1,0.948571,-0.948571,0.0000,-0.0000,0.6,-0.6,0.0,-0.0,0.0,-0.0
2,1.000000,-1.000000,0.4750,-0.4750,0.4,-0.4,0.0,-0.0,1.0,-1.0


,CoalA,CoalA-pos,GasA,GasA-pos,WindA,OilB,OilB-pos,CoalC,CoalC-pos
0,1.000000,0.000000,0.0125,0.9875,0.3,0.0,1.0,1.0,0.0
1,0.948571,0.051429,0.0000,1.0000,0.6,0.0,1.0,0.0,1.0
2,1.000000,0.000000,0.4750,0.5250,0.4,0.0,1.0,1.0,0.0


In [22]:
network.generators_t.p["CoalC"] / power_plants['Dafoeland']['CoalC']

marginal_costs["WindA"] - network.buses_t.marginal_price

,Dafoeland
0,-60.0
1,-30.0
2,-60.0
